In [110]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [111]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [112]:
# Example input data
input_data = {
    'CreditScore': 834,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 49,
    'Tenure': 2,
    'Balance': 131394.56,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 0,
    'EstimatedSalary': 194365.76
}

In [113]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/siddharth0807/Desktop/Python/annClassification_project/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [114]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,834,France,Female,49,2,131394.56,1,0,0,194365.76


In [115]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,834,France,0,49,2,131394.56,1,0,0,194365.76


In [116]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,834,0,49,2,131394.56,1,0,0,194365.76,1.0,0.0,0.0


In [117]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 1.8924039 , -1.09499335,  0.96044552, -1.04241787,  0.88528636,
        -0.91668767, -1.54035103, -1.02583358,  1.63320766,  1.00150113,
        -0.57946723, -0.57638802]])

In [118]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 27ms/step


array([[0.33056003]], dtype=float32)

In [119]:
prediction_proba = prediction[0][0]

In [120]:
prediction_proba

0.33056003

In [121]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
